In [1]:
# Modif target_binary, ignore_column for next participant ????????????????????????????
data_path = '/home/lea/Documents/obsidian_notes/masterADS/ads_thesis_RIFT'
trial_info_file = '/A0/converted/trialinfo_matrix_A0_cleanedtrials.mat'
data_file = '/A0/Experimental/data.mat'
epoch_file = '/my_code/epochs_from_matrix/epochs_0.fif'

relevant_electrodes = ['O2','PO4','PO8','P8','P4','P6','P2','P1','P3','P5','P7','PO7','PO3','O1','Pz','POz','Oz','Iz']
electrodes_kabir = ['POz', 'PO3', 'PO4', 'Pz', 'P1', 'P3', 'Oz', 'O1', 'O2']

# Dic to index trial_info matrix
info_colnames = {'0':0 ,'1':1 ,'RT':2,'PTB_check_RT_error':3, 'answer':4,'stimuli_target_interval':5,\
                      'base_delay':6, 'flicker_sides':7, 'cued_side':8, 'orients_L':9, 'orients_r':10,\
                      'ignore_column':11, 'target_angle':12, 'tagging_type':13, 'trial_number':14}

In [2]:
import scipy.io as sio
import scipy
import numpy as np
import pandas as pd
import eelbrain
import mne
from mne.time_frequency import EpochsTFRArray
import matplotlib.pyplot as plt
import copy


In [3]:
# Load trial info
f = sio.loadmat(f'{data_path}{trial_info_file}')
trial_info = f['all_info']


In [4]:
# Get target binary (i.e if a shift happened)
f = sio.loadmat(f'{data_path}{data_file}')
target_binary =f['data']['targets_binary'][0][0]
# Remove practice trials
target_binary = target_binary[1:,:]
# Collapse to trial indexes
original_shape = target_binary.shape
new_shape = (original_shape[0] * original_shape[1],) + original_shape[2:]
target_binary = np.reshape(target_binary, new_shape)
target_binary = pd.DataFrame({'trial_number':np.arange(target_binary.shape[0]), 'target_binary':target_binary})

# Create pd df to pass to mne metadata
pd_info = pd.DataFrame(trial_info, columns=info_colnames.keys())
pd_info['trial_number']= pd_info['trial_number'].astype(int) - 1
pd_info = pd.merge(pd_info, target_binary, on = 'trial_number')

# Load eeg epochs .fif and add metadata
epochs = mne.read_epochs(f'{data_path}{epoch_file}')
epochs.metadata = pd_info


/tmp/ipykernel_18849/4056453725.py:18: RuntimeWarning: This filename (/home/lea/Documents/obsidian_notes/masterADS/ads_thesis_RIFT/my_code/epochs_from_matrix/epochs_0.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(f'{data_path}{epoch_file}')


In [5]:
# Keep only posterior electrodes
epochs.pick_channels(relevant_electrodes)

Number of events,440
Events,1: 440
Time range,-1.000 – 2.299 s
Baseline,off


In [6]:
# Keep only trials where there was no target
epochs = epochs['target_binary==0']

# A. Frequency tagging condition
# 1 - Tag presence
## 1.1 FFT & SNR

In [7]:
epochs_freqtag = epochs['tagging_type==0'].copy()

In [8]:
# ATTNETION TOUTES LES EPOCHS PAS JUSTE FREQTAG
test = eelbrain.load.mne.epochs_ndvar(f'{data_path}{epoch_file}')

/home/lea/anaconda3/envs/ads_thesis/lib/python3.12/site-packages/eelbrain/_io/fiff.py:1173: RuntimeWarning: This filename (/home/lea/Documents/obsidian_notes/masterADS/ads_thesis_RIFT/my_code/epochs_from_matrix/epochs_0.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epochs, proj=proj)
/home/lea/anaconda3/envs/ads_thesis/lib/python3.12/site-packages/eelbrain/_io/fiff.py:1180: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  x = epochs.get_data()
